In [20]:
import pandas as pd
import numpy as np
import datetime
#import pandas_profiling # pip install pandas_profiling

taxiCancellationScore = pd.read_csv('Kaggle_YourCabs_score.csv')
taxiCancellationTraining = pd.read_csv('Kaggle_YourCabs_training.csv')

In [21]:
#taxiCancellationScore.Profile_Report()

In [22]:
taxiCancellationTraining.columns

Index(['id', 'user_id', 'vehicle_model_id', 'package_id', 'travel_type_id',
       'from_area_id', 'to_area_id', 'from_city_id', 'to_city_id', 'from_date',
       'to_date', 'online_booking', 'mobile_site_booking', 'booking_created',
       'from_lat', 'from_long', 'to_lat', 'to_long', 'Car_Cancellation',
       'Cost_of_error'],
      dtype='object')

In [23]:
taxiCancellationTraining

,id,user_id,vehicle_model_id,package_id,travel_type_id,from_area_id,to_area_id,from_city_id,to_city_id,from_date,to_date,online_booking,mobile_site_booking,booking_created,from_lat,from_long,to_lat,to_long,Car_Cancellation,Cost_of_error
0,132512,22177,28,NaN,2,83.0,448.0,NaN,NaN,1/1/2013 2:00,NaN,0,0,1/1/2013 1:39,12.924150,77.672290,12.927320,77.635750,0,1.0
1,132513,21413,12,NaN,2,1010.0,540.0,NaN,NaN,1/1/2013 9:00,NaN,0,0,1/1/2013 2:25,12.966910,77.749350,12.927680,77.626640,0,1.0
2,132514,22178,12,NaN,2,1301.0,1034.0,NaN,NaN,1/1/2013 3:30,NaN,0,0,1/1/2013 3:08,12.937222,77.626915,13.047926,77.597766,0,1.0
3,132515,13034,12,NaN,2,768.0,398.0,NaN,NaN,1/1/2013 5:45,NaN,0,0,1/1/2013 4:39,12.989990,77.553320,12.971430,77.639140,0,1.0
4,132517,22180,12,NaN,2,1365.0,849.0,NaN,NaN,1/1/2013 9:00,NaN,0,0,1/1/2013 7:53,12.845653,77.677925,12.954340,77.600720,0,1.0
5,132518,17712,12,NaN,2,1021.0,1323.0,NaN,NaN,1/1/2013 22:30,NaN,0,0,1/1/2013 8:01,13.028530,77.546250,12.869805,77.653211,0,1.0
6,132519,22172,12,NaN,1,571.0,NaN,15.0,108.0,1/1/2013 9:45,41275.40625,0,0,1/1/2013 9:21,12.951850,77.696420,NaN,NaN,0,1.0
7,132520,22181,12,NaN,2,1192.0,832.0,NaN,NaN,1/1/2013 11:00,NaN,0,0,1/1/2013 9:39,12.976770,77.572700,12.880190,77.645580,0,1.0
8,132521,22182,65,2.0,3,448.0,NaN,NaN,NaN,1/1/2013 16:00,NaN,0,0,1/1/2013 9:44,12.927320,77.635750,NaN,NaN,0,1.0
9,132522,22184,12,NaN,2,516.0,376.0,NaN,NaN,1/1/2013 11:00,NaN,0,0,1/1/2013 9:49,13.005600,77.657990,12.902450,77.660810,0,1.0


In [24]:
taxiCancellationTraining.isnull().sum()

id                         0
user_id                    0
vehicle_model_id           0
package_id             35881
travel_type_id             0
from_area_id              88
to_area_id              9138
from_city_id           27086
to_city_id             41843
from_date                  0
to_date                17890
online_booking             0
mobile_site_booking        0
booking_created            0
from_lat                  93
from_long                 93
to_lat                  9138
to_long                 9138
Car_Cancellation           0
Cost_of_error              0
dtype: int64

In [25]:
taxiCancellationScore.isnull().sum()

id                         0
user_id                    0
vehicle_model_id           0
package_id              8215
travel_type_id             0
from_area_id              20
to_area_id              2150
from_city_id            6288
to_city_id              9635
from_date                  0
to_date                 4127
online_booking             0
mobile_site_booking        0
booking_created            0
from_lat                  20
from_long                 20
to_lat                  2150
to_long                 2150
Unnamed: 18            10000
Unnamed: 19            10000
dtype: int64

#Convert Date String into Timestamp()
def date_time(y,x):
    from_date= [None]*y.shape[0]
    for i in range(y.shape[0]):
        if (type(y[x][i] is str and y[x][i].isnull() == False)):
            try:
                datetime_ob = datetime.datetime.strptime(y[x][i],"%m/%d/%Y %H:%M").weekday()
            except:
                print(int)
            y.loc[:,x].replace(to_replace=y.loc[:,x].values[i], value = datetime_ob, inplace = True)

In [26]:
#convert try 2
from_weekdate = [] #days of week
from_time = []
for x in taxiCancellationScore['from_date']:
    if type(x) is str:
        ziz = x.split(' ')
        day = ziz[0].split('/')
        for z in range(len(day)):
            day[z] = int(day[z])
        from_weekdate.append(datetime.date(day[2], day[0], day[1]).weekday())
        #from_weekdate is now ints as day of week, 0=sunday etc

        time = ziz[1].split(':')
        from_time.append(int(time[0])*60 + int(time[1]))
        #from_time is now ints as minute of day
        
        
booking_weekdate = [] #days of week
booking_time = []
for x in taxiCancellationScore['booking_created']:
    if type(x) is str:
        ziz = x.split(' ')
        day = ziz[0].split('/')
        for z in range(len(day)):
            day[z] = int(day[z])
        booking_weekdate.append(datetime.date(day[2], day[0], day[1]).weekday())
        #booking_weekdate is now ints as day of week, 0=sunday etc

        time = ziz[1].split(':')
        booking_time.append(int(time[0])*60 + int(time[1]))
        #booking_time is now ints as minute of day
        
#to be appended: from_weekdate, from_time, booking_weekdate, booking_time

In [27]:
feature_cols = ['vehicle_model_id', 'package_id', 'travel_type_id',
                'from_area_id', 'to_area_id', 'from_city_id', 'to_city_id', 
       'online_booking', 'mobile_site_booking',
       'from_lat', 'from_long', 'to_lat', 'to_long']

#Replace Trained Value in DataFrame
date_time(taxiCancellationScore,'from_date')
taxiCancellationScore.loc[:,'from_date']

date_time(taxiCancellationScore,'booking_created')
taxiCancellationScore.loc[:,'booking_created']

In [28]:
X = taxiCancellationTraining.loc[:,feature_cols] # Features
y = taxiCancellationTraining.loc[:,'Car_Cancellation'] # Target variable

In [29]:
X_test = taxiCancellationScore.loc[:,feature_cols] # Features
taxiCancellationTraining

,id,user_id,vehicle_model_id,package_id,travel_type_id,from_area_id,to_area_id,from_city_id,to_city_id,from_date,to_date,online_booking,mobile_site_booking,booking_created,from_lat,from_long,to_lat,to_long,Car_Cancellation,Cost_of_error
0,132512,22177,28,NaN,2,83.0,448.0,NaN,NaN,1/1/2013 2:00,NaN,0,0,1/1/2013 1:39,12.924150,77.672290,12.927320,77.635750,0,1.0
1,132513,21413,12,NaN,2,1010.0,540.0,NaN,NaN,1/1/2013 9:00,NaN,0,0,1/1/2013 2:25,12.966910,77.749350,12.927680,77.626640,0,1.0
2,132514,22178,12,NaN,2,1301.0,1034.0,NaN,NaN,1/1/2013 3:30,NaN,0,0,1/1/2013 3:08,12.937222,77.626915,13.047926,77.597766,0,1.0
3,132515,13034,12,NaN,2,768.0,398.0,NaN,NaN,1/1/2013 5:45,NaN,0,0,1/1/2013 4:39,12.989990,77.553320,12.971430,77.639140,0,1.0
4,132517,22180,12,NaN,2,1365.0,849.0,NaN,NaN,1/1/2013 9:00,NaN,0,0,1/1/2013 7:53,12.845653,77.677925,12.954340,77.600720,0,1.0
5,132518,17712,12,NaN,2,1021.0,1323.0,NaN,NaN,1/1/2013 22:30,NaN,0,0,1/1/2013 8:01,13.028530,77.546250,12.869805,77.653211,0,1.0
6,132519,22172,12,NaN,1,571.0,NaN,15.0,108.0,1/1/2013 9:45,41275.40625,0,0,1/1/2013 9:21,12.951850,77.696420,NaN,NaN,0,1.0
7,132520,22181,12,NaN,2,1192.0,832.0,NaN,NaN,1/1/2013 11:00,NaN,0,0,1/1/2013 9:39,12.976770,77.572700,12.880190,77.645580,0,1.0
8,132521,22182,65,2.0,3,448.0,NaN,NaN,NaN,1/1/2013 16:00,NaN,0,0,1/1/2013 9:44,12.927320,77.635750,NaN,NaN,0,1.0
9,132522,22184,12,NaN,2,516.0,376.0,NaN,NaN,1/1/2013 11:00,NaN,0,0,1/1/2013 9:49,13.005600,77.657990,12.902450,77.660810,0,1.0


## 4 Models

In [31]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout

#Logistic Regresstion

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')
logreg.fit(X, y)

In [33]:
#Deep Learning Keras Model
#relu -linear
#sigmoid function for final layer
#Dropout 50%
#Dimension input features column
#Output binary classification 0 and 1
model = Sequential()
model.add(Dense(124, activation='relu', input_dim = X.shape[1]))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.summary()
#Using binary_crossentropy
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 124)               1736      
_________________________________________________________________
dropout_4 (Dropout)          (None, 124)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8000      
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_6 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                

In [ ]:
#KNN
n_neigh = 3
from sklearn.neighbors import KNeighborsClassifier
neighbor = KNeighborsClassifier(n_neigh)
neighbor.fit(X, y) 

In [34]:
#Naive Bayes
from sklearn.naive_bayes import GaussianNB
gaus = GaussianNB()
gaus.fit(X, y)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

# Prediction

In [ ]:
model.fit(X,y, epochs=100, batch_size = 10, verbose = 1)

Epoch 1/100
43431/43431 [==============================] - 15s 351us/step - loss: nan - accuracy: 0.0000e+00
Epoch 2/100
43431/43431 [==============================] - 12s 286us/step - loss: nan - accuracy: 0.0000e+00
Epoch 3/100
43431/43431 [==============================] - 11s 246us/step - loss: nan - accuracy: 0.0000e+00
Epoch 4/100
41080/43431 [===========================>..] - ETA: 0s - loss: nan - accuracy: 0.0000e+00

In [ ]:
model.save('trained_model.h5')

In [ ]:
y_pred_logreg = logreg.predict(X_test)
y_pred_keras= model.predict(X_test)
y_pred_neigh= neighbor.predict(X_test)
y_pred_gaus= gaus.predict(X_test)

# Performance of Each Model

# visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
from sklearn import metrics
import numpy as np

cnf_matrix = metrics.confusion_matrix(y, y_pred_logreg)
cnf_matrix

In [ ]:
class_names=['Not play','Play'] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
# create heatmap
sns.heatmap(cnf_matrix, annot=True, cmap="YlGnBu" ,fmt='d', xticklabels=class_names, yticklabels=class_names)
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix')
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

